In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import json
import numpy as np
import torch
import pandas as pd
import tqdm
from torch.utils.data import Dataset, DataLoader
import glob
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import sys
sys.path = ['../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph
n_splits = 64
use_Hs = False
path = "../../results/pubchem/amd-server/noH/timeout60_4"
for i in range(n_splits):
    dname = glob.glob(path+"/%d/min_dfs_codes_split*.json"%(i+1))[0]
    didx = int(dname.split("split")[-1][:-5])
    with open(path+"/%d/min_dfs_codes_split%d.json"%(i+1, didx), 'r') as f:
        codes = json.load(f)
    d_dict = {}
    for smiles, code in tqdm.tqdm(codes.items()):
        d = smiles2graph(smiles, useHs=use_Hs, addLoops=False, skipCliqueCheck=False)
        data = {}
        data['x'] = d.x.detach().cpu().numpy().tolist()
        data['z'] = d.z.detach().cpu().numpy().tolist()
        data['edge_attr'] = d.edge_attr.detach().cpu().numpy().tolist()
        data['edge_index'] = d.edge_index.detach().cpu().numpy().tolist()
        d_dict[smiles] = data
    with open(path+"/%d/data_split%d.json"%(i+1, didx), 'w') as f:
        json.dump(d_dict, f)

 53%|███████████████████████████████████████████████████████████████████████████████████████▏                                                                              | 82041/156171 [02:52<02:36, 474.95it/s]


KeyboardInterrupt: 